<a href="https://colab.research.google.com/github/arpm92/Train-an-Object_Detection_Model_with_Keras/blob/main/R_CNN_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>